In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter('ignore')

In [ ]:
data = pd.read_csv(r'C:/Users/uzmap/Documents/GitHub/ForEx/processed_data.csv',index_col='Unnamed: 0')
data.head()

,Unnamed: 0.1,Date,Open_price,Day_high,Day_low,Closing_price,Currency Pair,Trend_Open_price,Seasonal_Open_price,Residual_Open_price,...,Seasonal_Day_low,Residual_Day_low,RSI,MACD,MACD_Signal,BB_Upper,BB_Lower,day_of_week,month,is_weekend
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0.000000,0,2014-11-07,0.386891,0.384657,0.389693,0.386954,USD/INR,0.394461,0.522182,0.60751,...,0.473615,0.538767,64.491363,0.000000,0.000000,0.391812,0.38654,4,11,0
0.000055,1,2014-11-10,0.387590,0.384752,0.389693,0.387558,USD/INR,0.394461,0.557718,0.60751,...,0.460915,0.538767,64.491363,0.000048,0.000010,0.391812,0.38654,0,11,0
0.000109,2,2014-11-11,0.387781,0.384248,0.390750,0.387641,USD/INR,0.394461,0.533733,0.60751,...,0.475153,0.538767,64.491363,0.000092,0.000026,0.391812,0.38654,1,11,0
0.000164,3,2014-11-12,0.387641,0.384280,0.389757,0.386897,USD/INR,0.394461,0.541190,0.60751,...,0.422273,0.538767,64.491363,0.000066,0.000034,0.391812,0.38654,2,11,0
0.000219,4,2014-11-13,0.386752,0.384676,0.389757,0.388003,USD/INR,0.394461,0.534029,0.60751,...,0.456504,0.538767,64.491363,0.000133,0.000054,0.391812,0.38654,3,11,0


LSTM

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Convert Date column to datetime (if necessary)
data['Date'] = pd.to_datetime(data['Date'])

# List of unique currency pairs
currency_pairs = data['Currency Pair'].unique()

# Function to create sequences
def create_sequences(data, seq_length=30):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1])  # Use all features except the target
        y.append(data[i+seq_length, -1])     # The target variable
    return np.array(X), np.array(y)

# Dictionary to store evaluation metrics for each currency pair
results = {}

# Hyperparameters
SEQ_LENGTH = 30  # Number of time steps in each sequence
EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.001

# Loop over each currency pair
for pair in currency_pairs:
    print(f"Training model for currency pair: {pair}")
    
    # Filter data for the current currency pair
    pair_data = data[data['Currency Pair'] == pair]
    
    # Sort by date to ensure proper time series order
    pair_data = pair_data.sort_values('Date')
    
    # Select features and target (Closing price is the target)
    features = pair_data[['Open_price', 'Day_high', 'Day_low', 'RSI', 'MACD', 'Trend_Open_price', 'Seasonal_Open_price', 'Residual_Open_price', 'BB_Upper', 'BB_Lower']]
    target = pair_data['Closing_price']
    
    # Combine features and target for sequence creation
    data_combined = pd.concat([features, target], axis=1).values
    
    # Split data into train and test sets (80-20 split)
    split_idx = int(len(data_combined) * 0.8)
    train_data = data_combined[:split_idx]
    test_data = data_combined[split_idx:]

    # Create sequences
    X_train, y_train = create_sequences(train_data, SEQ_LENGTH)
    X_test, y_test = create_sequences(test_data, SEQ_LENGTH)
    
    # Build LSTM model
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(SEQ_LENGTH, X_train.shape[2])),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error')
    
    # Train the model
    model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), verbose=1)
    
    # Make predictions
    y_pred_test = model.predict(X_test)
    
    # Calculate evaluation metrics
    mae = mean_absolute_error(y_test, y_pred_test)
    mape = mean_absolute_percentage_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)
    
    # Store the results for this currency pair
#     results[pair] = {
#         'MAE': mae,
#         'MAPE': mape,
#         'R^2': r2
#     }
    
#     print(f"Results for {pair} - MAE: {mae}, MAPE: {mape}, R^2: {r2}")

# # Display final results for each currency pair
# print("\nFinal Results for Each Currency Pair:")
# for pair, metrics in results.items():
#     print(f"{pair} - MAE: {metrics['MAE']}, MAPE: {metrics['MAPE']}, R^2: {metrics['R^2']}")
    
 
    # Store the results and predictions for this currency pair
    results[pair] = {
        'MAE': mae,
        'MAPE': mape,
        'R^2': r2,
        'y_true': y_test,       # True values for the test set
        'y_pred': y_pred_test    # Predicted values for the test set
    }
    
print(f"Results for {pair} - MAE: {mae}, MAPE: {mape}, R^2: {r2}")

# Display final results and predictions for each currency pair
print("\nFinal Results for Each Currency Pair:")
for pair, metrics in results.items():
    print(f"{pair} - MAE: {metrics['MAE']}, MAPE: {metrics['MAPE']}, R^2: {metrics['R^2']}")
    print(f"True values (sample): {metrics['y_true'][:5].flatten()}")
    print(f"Predicted values (sample): {metrics['y_pred'][:5].flatten()}\n")


Training model for currency pair: USD/INR
Epoch 1/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0144 - val_loss: 7.9136e-04
Epoch 2/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.0851e-04 - val_loss: 2.5073e-04
Epoch 3/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7.3650e-05 - val_loss: 8.5377e-05
Epoch 4/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.1960e-05 - val_loss: 3.6630e-05
Epoch 5/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.2048e-05 - val_loss: 3.6615e-05
Epoch 6/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.8058e-05 - val_loss: 2.4076e-05
Epoch 7/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.3521e-05 - val_loss: 2.1753e-05
Epoch 8/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.9302e-05 - val_loss: 2.1149e-05
Epoch 9/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.8886e-05 - val_loss: 1.6505e-05
Epoch 10/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.7719e-05 - val_loss: 1.5767e-05
Epoch 11/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s